In [375]:
# %config IPCompleter.greedy=True
# import libraries
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
# import requests
# from lxml import html as lh
# import traceback
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
# from timeit import default_timer as timer
import time

### Settings for browser

In [2]:
# create options object for chromium webdriver
options = webdriver.ChromeOptions()
# setup path for brave binary (executable file)
options.binary_location = "C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"
# add extensions for brave to open new instance of browser with VPN extension activated
# for this purpose I used extensions source code extractor on http://crxextractor.com/
# using link of extension found in store
options.add_extension(r"extension_4_1_0_0.crx")
# start chrome maximized
options.add_argument("start-maximized")
# downloaded chromium driver binaries and place it in project directory
chrome_driver_binary = "chromedriver.exe"

### Chrome Driver

In [457]:
# Chrome - controls browser
driver = webdriver.Chrome(chrome_driver_binary, options=options)
# choose which site you want to visit
driver.get("https://www.otodom.pl/sprzedaz/mieszkanie/wroclaw/?search%5Bfilter_float_price_per_m%3Afrom%5D=1&search%5Bfilter_float_price%3Afrom%5D=1&search%5Border%5D=filter_float_price%3Adesc&search%5Bcity_id%5D=39&nrAdsPerPage=24&page=254")
# check if it open right site by site title
assert "Mieszkanie na sprzedaż w Wrocław, dolnośląskie - www.otodom.pl" in driver.title
close_cookie_bar = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, "cookiesBarClose")))
close_cookie_bar.click()

#### Get page content and parse it with beautifulSoup

In [246]:
# attributes to collect
offer_id = []
offer_district = []
offer_price = []
offer_price_per_m2 = []
offer_room_number = []
offer_flat_surface_m2 = []

offer_floor = []
offer_floor_number = []
offer_prod_year = []
offer_rent = []
offer_market = []
offer_building_type = []
offer_finish_condition = []


In [458]:
# helpers
current_page = 1
page_number = 500
doubled_offers = 0

offer_row = {}
offer_overview_posibilities = ["Piętro:", "Liczba pięter:", "Rok budowy:", "Czynsz:", "Rynek:", "Rodzaj zabudowy:", 
                               "Stan wykończenia:"]

In [460]:
while current_page <= page_number:
    # get full page source
    content = driver.page_source
    
    # parse it with beautiflesoup
    soup = bs(content)
    
    #iterate through all articles (offer details are in articles)
    for offer_article in soup.find_all("article"):
        
        # if article not in history then...
        if offer_article["id"] not in offer_id:   

            # add offer id to history
            offer_id.append(offer_article["id"])
            
            # add offer price
            price = offer_article.find('li', attrs={'class':'offer-item-price'}).get_text()
            if price:
                offer_price.append(price.strip())
            else:
                offer_price.append(None)

            # add offer room number
            room_number = offer_article.find('li', attrs={'class':'offer-item-rooms hidden-xs'})
            if room_number:
                offer_room_number.append(room_number.get_text())
            else:
                offer_room_number.append(None)

            # add offer surface 
            flat_m2 = offer_article.find('li', attrs={'class':'hidden-xs offer-item-area'})
            if flat_m2:
                offer_flat_surface_m2.append(flat_m2.get_text())
            else:
                offer_flat_surface_m2.append(None)

            # add offer price per m2
            price_per_m2 = offer_article.find('li', attrs={'class':'hidden-xs offer-item-price-per-m'})
            if price_per_m2:
                offer_price_per_m2.append(price_per_m2.get_text())
            else:
                offer_price_per_m2.append(None)

            
            # add offer district
            district = offer_article.find('p', attrs={"class":"text-nowrap"})
            if district:
                offer_district.append(district.get_text())
            else:
                offer_district.append(None)

            # open new tab process
            # store current tab for return
            main_window = driver.current_window_handle
            
            # get URL of 
            offer_details_link = soup.find(attrs={"id":offer_article["id"]}).get("data-url")
            
            # open new tab and switch driver to it
            driver.execute_script("window.open();")
#             WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
            driver.switch_to.window(driver.window_handles[1])

            # set driver to get new offer details site and check if data is there
            driver.get(offer_details_link)
            
#             elif driver.find_element_by_tag_name("h2").text == "To ogłoszenie nie jest już dostępne":
#                 offer_id.pop()
#                 offer_district.pop()
#                 offer_price.pop()
#                 offer_price_per_m2.pop()
#                 offer_flat_surface_m2.pop()
#                 offer_room_number.pop()

#                 # close tab
#                 driver.close()

#                 # back to the main window
#                 driver.switch_to.window(main_window)

#                 continue

            # collect data from offer details site           
            # flat offer overview contains rest of attributes
            while True:
                try:
                    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, "css-dwmx8v-Fe")))
                    break
                except:
                    driver.refresh()

            # save overview element to variable
            overview = driver.find_element_by_class_name("css-dwmx8v-Fe")
            details = overview.find_elements_by_tag_name("li")

            # add all li text in list
            overview_categories = [category.text for category in details]

            # search for data in list by substring             
            floor = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[0] in s]
            floor_number = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[1] in s]
            production_year = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[2] in s]
            rent = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[3] in s]
            market = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[4] in s]
            building_type = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[5] in s]
            finish_condition = [s for i, s in enumerate(overview_categories) if offer_overview_posibilities[6] in s]

            if floor:
                offer_floor.append(floor[0])
            else:
                offer_floor.append(None)
            if floor_number:
                offer_floor_number.append(floor_number[0])
            else:
                offer_floor_number.append(None)
            if production_year:
                offer_prod_year.append(production_year[0])
            else:
                offer_prod_year.append(None)
            if rent:
                offer_rent.append(rent[0])
            else:
                offer_rent.append(None)
            if market:
                offer_market.append(market[0])
            else:
                offer_market.append(None)
            if building_type:
                offer_building_type.append(building_type[0])
            else:
                offer_building_type.append(None)
            if finish_condition:
                offer_finish_condition.append(finish_condition[0])
            else:
                offer_finish_condition.append(None)

            # close tab
            driver.close()
            WebDriverWait(driver, 5).until(EC.number_of_windows_to_be(1))
            # back to the main window
            driver.switch_to.window(main_window)
            
        else:
            doubled_offers += 1
            
    try:
        element = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, "pager-next")))
        element.click()
    except:
        print("Problem occured")
    
    print(f"Number of doubled offers: {doubled_offers}")    
    current_page += 1

Number of doubled offers: 6
Number of doubled offers: 9
Number of doubled offers: 12
Number of doubled offers: 15
Number of doubled offers: 19
Number of doubled offers: 22
Number of doubled offers: 25
Number of doubled offers: 28
Number of doubled offers: 31
Number of doubled offers: 34
Number of doubled offers: 37
Number of doubled offers: 40
Number of doubled offers: 42
Number of doubled offers: 47
Number of doubled offers: 50
Number of doubled offers: 52
Number of doubled offers: 55
Number of doubled offers: 59
Number of doubled offers: 62
Number of doubled offers: 65
Number of doubled offers: 68
Number of doubled offers: 71
Number of doubled offers: 74
Number of doubled offers: 78
Number of doubled offers: 80
Number of doubled offers: 83
Number of doubled offers: 86
Number of doubled offers: 89
Number of doubled offers: 93
Number of doubled offers: 96
Number of doubled offers: 100
Number of doubled offers: 106
Number of doubled offers: 110
Number of doubled offers: 113
Number of do

KeyboardInterrupt: 

In [462]:
# test = ['Powierzchnia: 82 m²', 'Liczba pokoi: 4', 'Rynek: wtórny', 'Rodzaj zabudowy: apartamentowiec', 
#         'Piętro: parter', 'Liczba pięter: 3', 'Materiał budynku: keramzyt', 'Okna: drewniane', 'Ogrzewanie: kotłownia', 
#         'Rok budowy: 2015', 'Stan wykończenia: do zamieszkania', 'Czynsz: 680 zł', 'Forma własności: pełna własność']

# indices = [s for i, s in enumerate(test) if offer_overview_posibilities[6] in s]
# indices
# x = driver.find_element_by_partial_link_text(offer_overview_posibilities[1])
# x
# print((offer_id))
# print((offer_district))
# print((offer_price))
# print((offer_price_per_m2))
# print((offer_floor))
# print((offer_floor_number))
# print((offer_prod_year))
# print((offer_rent))
# print((offer_market))
# print((offer_building_type))
# print((offer_finish_condition))
# print((offer_flat_surface_m2))
# print((offer_room_number))
offer_id.pop()
offer_district.pop()
offer_price.pop()
offer_price_per_m2.pop()
offer_flat_surface_m2.pop()
offer_room_number.pop()

# next_link = soup.find(attrs={"data-dir":"next"}).get("href")
# driver.find 


'3 pokoje'

In [465]:

print("id: ",len(offer_id))
print("distric: ",len(offer_district))
print("price: ",len(offer_price))
print("price per m2: ",len(offer_price_per_m2))
print("floor: ",len(offer_floor))
print("floor number: ",len(offer_floor_number))
print("prod year: ",len(offer_prod_year))
print("rent: ",len(offer_rent))
print("market: ",len(offer_market))
print("build type: ",len(offer_building_type))
print("finish cond: ",len(offer_finish_condition))
print("flat surface: ",len(offer_flat_surface_m2))
print("room number: ",len(offer_room_number))
offer_row = {"id":offer_id,"district":offer_district,"price":offer_price,"price_per_m2":offer_price_per_m2,
             "floor":offer_floor,"floor_number":offer_floor_number,"prod_year":offer_prod_year,"rent":offer_rent,
            "market":offer_market,"building_type":offer_building_type,"finish_condition":offer_finish_condition,
            "flat_surface_m2":offer_flat_surface_m2,"room_number":offer_room_number}

df = pd.DataFrame(offer_row)
df.tail(30)
# df.to_csv('flats_wroclaw_otodom.csv', index=False, encoding="utf32")
# current_page

id:  6853
distric:  6853
price:  6853
price per m2:  6853
floor:  6853
floor number:  6853
prod year:  6853
rent:  6853
market:  6853
build type:  6853
finish cond:  6853
flat surface:  6853
room number:  6853


,id,district,price,price_per_m2,floor,floor_number,prod_year,rent,market,building_type,finish_condition,flat_surface_m2,room_number
6823,offer-item-ad_id48WgA,"Mieszkanie na sprzedaż: Wrocław, Krzyki, Południe","451 619,37 zł",9 550 zł/m²,Piętro: 1,Liczba pięter: 6,Rok budowy: 2020,None,Rynek: pierwotny,Rodzaj zabudowy: blok,Stan wykończenia: do wykończenia,"47,29 m²",3 pokoje
6824,offer-item-ad_id48MVk,"Mieszkanie na sprzedaż: Wrocław, Krzyki, Klecina",451 528 zł,8 950 zł/m²,Piętro: 2,Liczba pięter: 3,Rok budowy: 2020,Czynsz: 350 zł,Rynek: pierwotny,Rodzaj zabudowy: blok,Stan wykończenia: do wykończenia,"50,45 m²",3 pokoje
6825,offer-item-ad_id46KrT,"Mieszkanie na sprzedaż: Wrocław, Krzyki, Księż...","451 515,60 zł",6 984 zł/m²,Piętro: 1,None,Rok budowy: 2018,None,Rynek: pierwotny,None,None,"64,65 m²",3 pokoje
6826,offer-item-ad_id477an,"Mieszkanie na sprzedaż: Wrocław, Śródmieście",451 468 zł,11 800 zł/m²,Piętro: 2,Liczba pięter: 8,Rok budowy: 2018,None,Rynek: pierwotny,Rodzaj zabudowy: blok,Stan wykończenia: do wykończenia,"38,26 m²",2 pokoje
6827,offer-item-ad_id48AJQ,"Mieszkanie na sprzedaż: Wrocław, Śródmieście, ...",451 468 zł,11 800 zł/m²,Piętro: 2,Liczba pięter: 8,Rok budowy: 2020,Czynsz: 8 zł,Rynek: pierwotny,Rodzaj zabudowy: apartamentowiec,None,"38,26 m²",2 pokoje
6828,offer-item-ad_id48i82,"Mieszkanie na sprzedaż: Wrocław, Nadodrze",451 468 zł,11 800 zł/m²,Piętro: 1,Liczba pięter: 8,Rok budowy: 2020,None,Rynek: pierwotny,Rodzaj zabudowy: apartamentowiec,None,"38,26 m²",2 pokoje
6829,offer-item-ad_id48B1e,"Mieszkanie na sprzedaż: Wrocław, Śródmieście",451 468 zł,11 800 zł/m²,Piętro: 2,Liczba pięter: 6,Rok budowy: 2020,None,Rynek: pierwotny,Rodzaj zabudowy: blok,None,"38,26 m²",2 pokoje
6830,offer-item-ad_id48Guz,"Mieszkanie na sprzedaż: Wrocław, Krzyki, Borek",451 468 zł,11 800 zł/m²,Piętro: 2,Liczba pięter: 8,Rok budowy: 2020,None,Rynek: pierwotny,Rodzaj zabudowy: blok,None,"38,26 m²",2 pokoje
6831,offer-item-ad_id48GVC,"Mieszkanie na sprzedaż: Wrocław, Nadodrze",451 468 zł,11 800 zł/m²,Piętro: 2,Liczba pięter: 8,Rok budowy: 2020,None,Rynek: pierwotny,Rodzaj zabudowy: apartamentowiec,None,"38,26 m²",2 pokoje
6832,offer-item-ad_id48W4a,"Mieszkanie na sprzedaż: Wrocław, Śródmieście, ...",451 468 zł,11 800 zł/m²,Piętro: 2,Liczba pięter: 8,Rok budowy: 2019,Czynsz: 6 zł,Rynek: pierwotny,Rodzaj zabudowy: blok,None,"38,26 m²",2 pokoje


In [ ]:
offer_districts_examples = {"Stare Miasto":["Szczepin", 
                                            "Stare Miasto", 
                                            "Przedmieście Świdnickie"], 
                            "Psie Pole":["Karłowice - Różanka", 
                                         "Osobowice - Rędzin", 
                                         "Lipa Piotrowska", 
                                         "Świniary", 
                                         "Widawa", 
                                         "Polanowice - Poświętne - Ligota",
                                         "Sołtysowice", 
                                         "Kowale", 
                                         "Strachocin - Swojczyce - Wojnów", 
                                         "Psie Pole - Zawidawie", 
                                         "Pawłowice"], 
                            "Krzyki":["Powstańców Śląskich", 
                                      "Huby", 
                                      "Tarnogaj", 
                                      "Przedmieście Oławskie", 
                                      "Księże", 
                                      "Brochów",
                                      "Bieńkowice", 
                                      "Gaj", 
                                      "Borek", 
                                      "Klecina", 
                                      "Krzyki - Partynice", 
                                      "Jagodno", 
                                      "Wojszyce", 
                                      "Ołtaszyn"], 
                            "Fabryczna":["Pracze Odrzańskie", 
                                         "Leśnica", 
                                         "Jerzmanowo - Jarnołtów - Strachowice - Osiniec", 
                                         "Maślice", 
                                         "Żerniki", 
                                         "Pilczyce - Kozanów - Popowice Płn.", 
                                         "Gądów - Popowice Płd.", 
                                         "Kuźniki", 
                                         "Nowy Dwór", 
                                         "Muchobór Wielki", 
                                         "Muchobór Mały", 
                                         "Gajowice", 
                                         "Grabiszyn - Grabiszynek", 
                                         "Oporów"], 
                            "Śródmieście":["Kleczków", 
                                           "Nadodrze", 
                                           "Ołbin", 
                                           "Plac Grunwaldzki", 
                                           "Zacisze - Zalesie - Szczytniki", 
                                           "Biskupin - Sępolno - Dąbie - Bartoszowice"]}


## Scrap table data from gp-sms.pl
### Based on article from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [ ]:
options = webdriver.ChromeOptions()
options.binary_location = "C:/Program Files (x86)/BraveSoftware/Brave-Browser/Application/brave.exe"
chrome_driver_binary = "chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, options=options)
server_name = []
gold_amount = []

# elem = driver.find_element_by_class_name("table-gold-available")
driver.get("http://gp-sms.pl/")
content = driver.page_source
soup = bs(content)
doc = lh.fromstring(content)
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    name=t.text_content()
    col.append((name,[]))

#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i > 0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [ ]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [ ]:
df.to_excel('gold_amount_gp_sms_scrap2.xlsx', index=False, encoding='utf-8')

In [ ]:
df.to_csv('gold_amount_gp_sms_scrap.csv', index=False, encoding='utf-8')

In [ ]:
df = pd.read_excel("gold_amount_gp_sms_scrap.xlsx", dtype={'Ilość CC': float})
df.to_excel('gold_amount_gp_sms_scrap2.xlsx', index=False, encoding='utf-8')

In [ ]:
df.head(100)

## Tutorial from selenium docs on custom example

In [ ]:
driver.get("http://gp-sms.pl/")
assert "Tibia GP za SMS" in driver.title
elem = driver.find_element_by_id("searchbar-gold-available")
elem.clear()
elem.send_keys("Harmonia")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source

## Tutorial from https://selenium-python.readthedocs.io/

In [ ]:
options = webdriver.ChromeOptions()
options.binary_location = "C:/Program Files (x86)/BraveSoftware/Brave-Browser/Application/brave.exe"
chrome_driver_binary = "chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, options=options)
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("dsafdaga")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
# driver.close()